In [1]:
from scipy.stats import fisher_exact
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import cross_val_score, StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from util.decoding.gee import GEEEstimator

from bids import BIDSLayout

In [2]:
BIDS_ROOT = 'bids'
DERIV_ROOT = os.path.join(BIDS_ROOT, 'derivatives')

In [3]:
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
events_fpaths = layout.get(
    scope = 'raw', 
    suffix = 'events', 
    extension = 'tsv', 
    return_type = 'filename')
events_dfs = [pd.read_csv(f, sep = '\t') for f in events_fpaths]
for i, df in enumerate(events_dfs):
    df['subject'] = i
events = pd.concat(events_dfs)
events = events.dropna()
events.head()

/home/johnv/anaconda3/envs/neuroforecasting/lib/python3.9/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


,onset,duration,sample,funded,choice,category,project_id,trial_type,subject
0,28.283,0,28283,1,0.0,place,PR81,image,0
1,46.266,0,46266,0,0.0,place,PR59,image,0
2,64.249,0,64249,1,0.0,face,PR11,image,0
4,98.266,0,98266,1,0.0,place,PR63,image,0
5,116.266,0,116266,0,0.0,place,PR53,image,0


In [4]:
funded = pd.Categorical(events.funded).to_numpy()
indiv_choice = pd.Categorical(events.choice).to_numpy()
stims = LabelEncoder().fit_transform(events.project_id)
subs = LabelEncoder().fit_transform(events.subject)

In [5]:
def ttest_1samp_kx2cv(scores, chance = 0.5):
    '''
    Input: list of k (2, n_tests) arrays of scores obtained from
            a series of k different 2-fold cross validation splits
            
    Output: t-values, p-values, and standard errors from 
            a kx2 t-test for classification metrics with null
            hypothesis that metric == 0.5. These are all
            (n_tests,) arrays. Test described in [1].
            
    References
    ----------
    [1] Dietterich TG (1998) Approximate Statistical Tests for 
        Comparing Supervised Classification Learning Algorithms.
        Neural Comput 10:1895–1923.
    '''
    from scipy.stats import t as t_dist
    above_chance = [(scrs - chance) for scrs in scores]
    if above_chance[0].ndim == 1:
        above_chance = [ac[:, np.newaxis] for ac in above_chance]
    # estimate standard error from all the splits
    s2 = [np.square(scrs - scrs.mean(0)).sum(0) for scrs in above_chance]
    s2 = np.stack(s2, axis = 0)
    # but only use first split for observed value since scores on CV splits
    p1 = above_chance[0][1, :] # are not truly independent of one another.
    se = np.sqrt(s2.mean(0)) # see [1]
    # after that, it's just a normal t-test
    t = p1 / se
    p = t_dist.sf(t, df = s2.shape[0]) # one-sided p-value 
    if t.shape == (1,):
        t, p, p1, se = t[0], p[0], p1[0], se[0]
    return t, p, (p1 + chance), se

## Cross-Validated Classification Score

We test whether we can predict whether a stimulus will be funded (aggregate) from individual choice behavior on a trial-by-trial basis using the same cross-validation scheme as we do for neural decoding. Note that a negative result doesn't mean these two things aren't related; they obviously are related given sufficient power, since individual and aggregate choice become practically the same thing when _n_ increases to become a substantial fraction of the population. However, if we can't get robustly above-chance cross-validated AUC given perfect information about individual choice behavior, we can assume that the incomplete individual choice information our aggregate choice decoder has access to does not explain its above-chance decoding accuracy.

In [6]:
# construct classifier
logistic_reg = GEEEstimator(family = 'binomial', cov_type = 'naive')

# cross-validate over stimuli
cv_methds = [StratifiedGroupKFold(2, shuffle = True, random_state = i) for i in range(10)]
fit_params = dict(groups = subs, maxiter = 500, params_niter = 25)
scores = [
    cross_val_score(
        logistic_reg,
        indiv_choice, funded,
        groups = stims,
        fit_params = fit_params,
        scoring = 'roc_auc',
        cv = cv) for cv in cv_methds
]

# test against null hpyothesis that AUC == 0.5
t, p, m, se = ttest_1samp_kx2cv(scores)
print('ROC-AUC is {:.2f} +/- {:.2f} ----> t({:d}) = {:.2f}, p = {:.2f}'.format(
    m, 1.96*se, len(scores), t, p
    )
)

ROC-AUC is 0.52 +/- 0.06 ----> t(10) = 0.52, p = 0.31


## In-Sample Regression Analysis

We can also do a more traditional regression analysis (without cross-validation), since the logistic classifier we use for decoding is just a run-of-the-mill Generalized Linear Model at the end of the day.

In [7]:
logistic_reg.fit(
    indiv_choice, funded, 
    **fit_params
)
print(logistic_reg.summary())

                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                 1605
Model:                                 GEE   No. clusters:                       18
Method:                        Generalized   Min. cluster size:                  85
                      Estimating Equations   Max. cluster size:                  91
Family:                           Binomial   Mean cluster size:                89.2
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Tue, 07 Jun 2022   Scale:                           1.000
Covariance type:                     naive   Time:                         10:22:38
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0025      0.018     -0.140      0.889      -0.038       0.033
x1     